<a href="https://colab.research.google.com/github/yudam-J/KDC-classification-with-BERT/blob/main/KDC_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import sys
import urllib.request
import json
import re
import numpy as np
from google.colab import drive
from glob import glob

drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/work_dir')

pd.set_option('display.float_format', '{:.0f}'.format)

Mounted at /content/drive/


In [ ]:
def get_book_descriptions(df, isbn_column, client_id, client_secret):
    """
    네이버 책 API를 통해 ISBN 번호를 바탕으로 책 설명을 가져오는 함수.

    Parameters:
    - df (pd.DataFrame): ISBN 번호가 있는 데이터프레임
    - isbn_column (str): ISBN 번호가 포함된 열의 이름
    - client_id (str): 네이버 API 클라이언트 ID
    - client_secret (str): 네이버 API 클라이언트 시크릿

    Returns:
    - pd.DataFrame: description 열이 추가된 데이터프레임
    """
    descriptions = []

    for isbn in df[isbn_column]:
        query = urllib.parse.quote(isbn)

        # 네이버 API URL
        url = f"https://openapi.naver.com/v1/search/book_adv?d_isbn={query}&display=1&start=1&sort=sim"

        # 요청 헤더 설정
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)

        try:
            # API 요청 보내기
            response = urllib.request.urlopen(request)
            rescode = response.getcode()

            if rescode == 200:
                # 응답 본문 읽기
                response_body = response.read()
                items = json.loads(response_body.decode('utf-8')).get("items", [])

                # HTML 태그 제거용 정규 표현식
                remove_tag = re.compile('<.*?>')

                # 책의 description 가져오기 (첫 번째 검색 결과)
                description = re.sub(remove_tag, '', items[0]['description']) if items else "No description available"
                descriptions.append(description)
            else:
                print(f"Error Code: {rescode}")
                descriptions.append("Error in fetching description")

        except urllib.error.HTTPError as e:
            print(f"HTTP Error: {e.code}")
            descriptions.append("HTTP Error")
        except urllib.error.URLError as e:
            print(f"URL Error: {e.reason}")
            descriptions.append("URL Error")
        except Exception as e:
            print(f"General Error: {str(e)}")
            descriptions.append("General Error")

    # DataFrame에 descriptions 열 추가
    df['description'] = descriptions

    return df

In [ ]:
df = pd.read_csv("NL_BO_BOOK_PUB_202408-1.csv", low_memory=False)
df = df[["TITLE_NM", "ISBN_THIRTEEN_NO", "SGVL_ISBN_NO",
         "ISBN_THIRTEEN_ORGT_NO", "KDC_NM"]]
df = df.dropna(subset=['KDC_NM'])
df = df[["TITLE_NM", "ISBN_THIRTEEN_NO", "KDC_NM"]]
df = df.dropna(subset=['ISBN_THIRTEEN_NO'])
df = df.drop_duplicates(subset=['ISBN_THIRTEEN_NO'])

In [ ]:
# 100개씩 나누기
df_chunks = np.array_split(df, len(df) // 100 + 1)

# 결과 확인
for i, chunk in enumerate(df_chunks):
    print(f"Chunk {i+1}:")
    print(chunk)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
...                   ...              ...    ...
780718  킹피셔 어린이 세계사 백과사전.    9788970574752    909
780719  킹피셔 어린이 세계사 백과사전.    9788970574769    909
780720  킹피셔 어린이 세계사 백과사전.    9788970574776    909
780721  킹피셔 어린이 세계사 백과사전.    9788970574783    909
780722  킹피셔 어린이 세계사 백과사전.    9788970574790    909

[100 rows x 3 columns]
Chunk 2672:
                     TITLE_NM ISBN_THIRTEEN_NO KDC_NM
780729  일본이 바다에 가라앉을지도 모른다구요？    9788952919021    445
780734                그림이랑 놀자    9788990627728    650
780738                까마귀 알퐁스    9788989427216    853
780742               나는 쇠무릎이야    9788988578988  813.8
780745                오줌 쏴아아！    9788901025858    210
...                       ...              ...    ...
780975                      말    9788901040622    408
780982                비주얼 박물관    9788901029702    010
781010            고향을 지키는 아이들    9788936440091  813.8
781014                 작은 그림책    9788936440176  813.8
781018               나는 둥그배미야    9

In [ ]:
for i in range(631, 700):
    chunk_with_descriptions = get_book_descriptions(df_chunks[i], 'ISBN_THIRTEEN_NO', '4ZGwbwRE6gbsvIrE2ogN', 'Ya1mE4urmK')
    file_path = f'/content/drive/MyDrive/work_dir/chunk{i}.csv'
    chunk_with_descriptions.to_csv(file_path, index=False)

In [ ]:
for i in range(700, 750):
    chunk_with_descriptions = get_book_descriptions(df_chunks[i], 'ISBN_THIRTEEN_NO', '4ZGwbwRE6gbsvIrE2ogN', 'Ya1mE4urmK')
    file_path = f'/content/drive/MyDrive/work_dir/chunk{i}.csv'
    chunk_with_descriptions.to_csv(file_path, index=False)

In [ ]:
for i in range(750, 1000):
    chunk_with_descriptions = get_book_descriptions(df_chunks[i], 'ISBN_THIRTEEN_NO', '4ZGwbwRE6gbsvIrE2ogN', 'Ya1mE4urmK')
    file_path = f'/content/drive/MyDrive/work_dir/chunk{i}.csv'
    chunk_with_descriptions.to_csv(file_path, index=False)

HTTP Error: 500
HTTP Error: 500
HTTP Error: 500
HTTP Error: 500


In [ ]:
for i in range(1000, 1500):
    chunk_with_descriptions = get_book_descriptions(df_chunks[i], 'ISBN_THIRTEEN_NO', '4ZGwbwRE6gbsvIrE2ogN', 'Ya1mE4urmK')
    file_path = f'/content/drive/MyDrive/work_dir/chunk{i}.csv'
    chunk_with_descriptions.to_csv(file_path, index=False)

Streaming output truncated to the last 5000 lines.
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP Error: 429
HTTP 